Convolutional Layer with Keras

In [ ]:
from sklearn.datasets import load_sample_images
import tensorflow as tf

images = load_sample_images()["images"]
images = tf.keras.layers.CenterCrop(height =70, width = 120)(images)
images = tf.keras.layers.Rescaling(scale=1 / 255)(images)
print(images.shape)
# original images 427x640
# 4D(trensor) 2 batch size, image size now 70*120 . 3 rgb number of feature maps 

2D convulutional layer (32 filters, 7x7 kernel)

In [ ]:
conv_layer = tf.keras.layers.Conv2D(filters=32,kernel_size=7,padding="same")
fmaps = conv_layer(images)
print(fmaps.shape)
# 2 images, 64x144 size, 32 feature maps paddig ="valid"
# 2 images 70x120 size, 32 feature maps paddig ="same" with padding

In [ ]:
kernels, biases = conv_layer.get_weights()
print(kernels.shape)
print(biases.shape)
# (7,7,3,32) kernel_height,kernel_width,input_channels(feature maps),output_channels

Pooling Layers (max number) 2x2 kerners and get a max number, no padding

In [ ]:
max_pool = tf.keras.layers.MaxPool2D(pool_size=2)

Reduce channels Depthwise max pooling

In [ ]:
class DepthPool(tf.keras.layers.Layer):
    def __init__(self, pool_size=2, **kwargs):
        super().__init__(**kwargs)
        self.pool_size = pool_size
        
    def call(self, inputs):
        shape = tf.shape(inputs) #shape[-1] number of channel
        groups = shape[-1] // self.pool_size #numbers of the channel grups
        new_shape = tf.concat([shape[:-1], [groups, self.pool_size]], axis=0)
        return tf.reduce_max(tf.reshape(inputs, new_shape), axis=-1)

Average Pooling (avg number)

In [ ]:
global_avg_pool = tf.keras.layers.GlobalAveragePooling2D()

Basic CNN

In [ ]:
from functools import partial

DefaultConv2D = partial(tf.keras.layers.Conv2D, kerner_size=3, padding="same", 
                        activation = "relu", kernel_initalizer="he_normal")

model = tf.keras.Sequential([
    DefaultConv2D(filters=64,kernel_size=7,input_shape=[28,28,1]),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation="relu", 
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=64, activation="relu", 
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=10,activation="softmax")
 
])